# 특성 선택기로 LLM 사용하기

##### 이 노트북은 애저 클라우드 인스턴스, 파이토치 2.4.1, 트랜스포머스 4.44.2, 라이트닝 2.3.3 버전을 사용해 실행했습니다.

특성 기반 방법에서는 사전 훈련된 트랜스포머의 임베딩을 사용해 사이킷런의 랜덤 포레스트와 로지스틱 회귀 모델을 훈련합니다.

<img src="https://github.com/rickiepark/MLQandAI/blob/main/supplementary/q18-using-llms/01_classifier-finetuning/figures/1_feature-based.png?raw=1" width=500>

In [1]:
# pip install transformers datasets lightning

In [2]:
# pip install watermark

%load_ext watermark
%watermark -p torch,transformers,datasets,sklearn

torch       : 2.4.1
transformers: 4.44.2
datasets    : 3.0.0
sklearn     : 0.22.1



In [3]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# 1 데이터셋 로드하기

In [4]:
import os.path as op

from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
import torch

from sklearn.feature_extraction.text import CountVectorizer

from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
from local_dataset_utilities import IMDBDataset

In [5]:
download_dataset()

df = load_dataset_into_to_dataframe()
partition_dataset(df)

100% | 80.23 MB | 5.13 MB/s | 15.63 sec elapsedClass distribution:


100%|██████████| 50000/50000 [19:32<00:00, 42.63it/s]


In [6]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")

# 2 토큰화

In [7]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv",
    },
)

print(imdb_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10000
    })
})


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased",
                                          clean_up_tokenization_spaces=False)
print("토크나이저의 최대 입력 길이:", tokenizer.model_max_length)
print("토크나이저의 어휘 사전 크기:", tokenizer.vocab_size)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

토크나이저의 최대 입력 길이: 512
토크나이저의 어휘 사전 크기: 30522


In [9]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [10]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
del imdb_dataset

# 3 DistilBERT를 특성 추출기로 사용하기

In [12]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device);

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [13]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [14]:
test_batch = {"attention_mask": imdb_tokenized["train"][:3]["attention_mask"].to(device),
              "input_ids": imdb_tokenized["train"][:3]["input_ids"].to(device)}

with torch.inference_mode():
    test_output = model(**test_batch)

test_output.last_hidden_state.shape

torch.Size([3, 512, 768])

In [15]:
cls_token_output = test_output.last_hidden_state[:, 0]
cls_token_output.shape

torch.Size([3, 768])

In [16]:
@torch.inference_mode()
def get_output_embeddings(batch):
    output = model(
        batch["input_ids"].to(device),
        attention_mask=batch["attention_mask"].to(device)).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}

In [17]:
import time
start = time.time()

imdb_features = imdb_tokenized.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
X_train = np.array(imdb_features["train"]["features"])
y_train = np.array(imdb_features["train"]["label"])

X_val = np.array(imdb_features["validation"]["features"])
y_val = np.array(imdb_features["validation"]["label"])

X_test = np.array(imdb_features["test"]["features"])
y_test = np.array(imdb_features["test"]["label"])

# 4 임베딩(추출된 특성)으로 모델 훈련하기

In [19]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

print("훈련 정확도", clf.score(X_train, y_train))
print("검증 정확도", clf.score(X_val, y_val))
print("테스트 정확도", clf.score(X_test, y_test))

end = time.time()
elapsed = end - start
print(f"소요 시간 {elapsed/60:.2f} min")

훈련 정확도 0.8879714285714285
검증 정확도 0.8852
테스트 정확도 0.876
소요 시간 123.90 min


In [20]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

print("훈련 정확도", clf.score(X_train, y_train))
print("검증 정확도", clf.score(X_val, y_val))
print("테스트 정확도", clf.score(X_test, y_test))

훈련 정확도 1.0
검증 정확도 0.8392
테스트 정확도 0.8274
